In [9]:
import pandas as pd
import numpy as np

# === 路径设置 ===
summary_path = "summary.csv"
prob_path = "neutrino_probabilities_full.csv"
output_path = "summary_with_real_significance_and_prob.csv"  # 修改输出文件名以区分

# === 读取数据 ===
df_summary = pd.read_csv(summary_path)
df_prob = pd.read_csv(prob_path)

# === 添加新列 ===
real_sig_min_list = []
real_sig_max_list = []
prob_low_list = []  # 存储 Probability (Low)
prob_high_list = []  # 存储 Probability (High)

for idx, row in df_summary.iterrows():
    erange = row["energy_range"].replace("_", "-")  # 将下划线替换为连字符
    try:
        # 查找中微子反应概率
        prob_row = df_prob[df_prob["EnergyAngle"] == erange]
        if prob_row.empty:
            print(f"[WARN] 未找到能段 {erange} 的中微子概率数据，填充 NaN")
            real_sig_min_list.append(np.nan)
            real_sig_max_list.append(np.nan)
            prob_low_list.append(np.nan)
            prob_high_list.append(np.nan)
            continue

        prob_low = prob_row["Probability (Low)"].values[0]
        prob_high = prob_row["Probability (High)"].values[0]

        # 获取其他必要参数
        tpr = row.get("Q_max_TPR", np.nan)
        fpr = row.get("Q_max_FPR", np.nan)
        n_nu = row.get("N_nu", np.nan)
        n_p = row.get("N_p", np.nan)

        if np.isnan(tpr) or np.isnan(fpr) or np.isnan(n_nu) or np.isnan(n_p):
            real_sig_min_list.append(np.nan)
            real_sig_max_list.append(np.nan)
            prob_low_list.append(np.nan)
            prob_high_list.append(np.nan)
            continue

        # 计算显著性上下限
        sig_min = n_nu * prob_low * tpr / np.sqrt(n_p * fpr + 1e-8)
        sig_max = n_nu * prob_high * tpr / np.sqrt(n_p * fpr + 1e-8)

        real_sig_min_list.append(sig_min)
        real_sig_max_list.append(sig_max)
        prob_low_list.append(prob_low)
        prob_high_list.append(prob_high)
    except Exception as e:
        print(f"[ERROR] 在处理 {erange} 时出错：{e}")
        real_sig_min_list.append(np.nan)
        real_sig_max_list.append(np.nan)
        prob_low_list.append(np.nan)
        prob_high_list.append(np.nan)

# === 添加到 summary 表中并保存 ===
df_summary["Significance_real_min"] = real_sig_min_list
df_summary["Significance_real_max"] = real_sig_max_list
df_summary["Neutrino_Probability_Low"] = prob_low_list  # 新增列：低能段概率
df_summary["Neutrino_Probability_High"] = prob_high_list  # 新增列：高能段概率

df_summary.to_csv(output_path, index=False)
print(f"✅ 已保存含真实显著性及中微子概率的新文件：{output_path}")



✅ 已保存含真实显著性及中微子概率的新文件：summary_with_real_significance_and_prob.csv
